In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from PIL import Image
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.metrics import MeanAbsoluteError
from sklearn.metrics import mean_absolute_error

In [3]:
image_directory = r"C:\Users\Dell\Desktop\Studies\Subjects\Deep Learning\assignments\assignment2\bau-ain3002-spring23-a2p1\images\images"
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [4]:
def createdataset(dataframe, image_direc, size=(224, 224)):
    image_list = []
    for img_id in dataframe['img_id']:
        full_path = os.path.join(image_direc, f"{img_id}.jpeg")
        img = cv2.imread(full_path)
        img_resized = cv2.resize(img, size)
        image_list.append(img_resized)
    return np.array(image_list) / 255.0

In [5]:
train_data = pd.read_csv(test_data)

train_targets = train_data['metastasis_ratio'].values

train_img_data = createdataset(train_data, image_directory)

X_training, X_validation, y_training, y_validation = train_test_split(train_img_data, train_targets, test_size=0.2, random_state=42)

test_data = pd.read_csv(test_data)
test_img_data = createdataset(test_data, image_directory)

In [7]:
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential([
    base_model ,

    layers.Flatten() , 
    layers.Dense(256 , kernel_regularizer=regularizers.l2(0.001) , activation='relu') ,
    layers.BatchNormalization() ,
    layers.Dropout(0.5) ,

      layers.Flatten() , 
    layers.Dense(256 , kernel_regularizer=regularizers.l2(0.001) , activation='relu') ,
    layers.BatchNormalization() ,
    layers.Dropout(0.5) ,

    layers.Flatten() , 
    layers.Dense(256 , kernel_regularizer=regularizers.l2(0.001) , activation='relu') ,
    layers.BatchNormalization() ,
    layers.Dropout(0.5) ,

        layers.Flatten() , 
    layers.Dense(256 , kernel_regularizer=regularizers.l2(0.001) , activation='relu') ,
    layers.BatchNormalization() ,
    layers.Dropout(0.5) ,

    layers.Dense(1, activation='linear')

])

model.summary()

model.compile(loss='mean_absolute_error', optimizer=RMSprop(learning_rate=0.0001), metrics=[MeanAbsoluteError()])



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 flatten (Flatten)           (None, 50176)             0         
                                                                 
 dense (Dense)               (None, 256)               12845312  
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                        

In [10]:
fittedmodel  = model.fit(X_training, y_training, 
                    validation_data=(X_validation, y_validation), 
                    batch_size=32, 
                    epochs=20)

Epoch 1/20
32/32 [==============================] - 12s 319ms/step - loss: 2.0470 - mean_absolute_error: 1.0090 - val_loss: 1.3582 - val_mean_absolute_error: 0.3246
Epoch 2/20
32/32 [==============================] - 5s 148ms/step - loss: 2.0255 - mean_absolute_error: 0.9956 - val_loss: 1.3485 - val_mean_absolute_error: 0.3228
Epoch 3/20
32/32 [==============================] - 4s 132ms/step - loss: 2.0327 - mean_absolute_error: 1.0107 - val_loss: 1.3488 - val_mean_absolute_error: 0.3309
Epoch 4/20
32/32 [==============================] - 5s 155ms/step - loss: 2.0038 - mean_absolute_error: 0.9895 - val_loss: 1.3284 - val_mean_absolute_error: 0.3180
Epoch 5/20
32/32 [==============================] - 5s 148ms/step - loss: 1.9922 - mean_absolute_error: 0.9853 - val_loss: 1.3493 - val_mean_absolute_error: 0.3463
Epoch 6/20
32/32 [==============================] - 4s 132ms/step - loss: 1.9631 - mean_absolute_error: 0.9635 - val_loss: 1.3432 - val_mean_absolute_error: 0.3473
Epoch 7/20
32/3

In [11]:
y_pred = model.predict(X_validation)

mae_score = mean_absolute_error(y_validation, y_pred)

print('MAE score:', mae_score)

9/9 [==============================] - 1s 102ms/step
MAE score: 0.296849672138691


In [12]:
y_pred = model.predict(test_img_data)

submission = pd.DataFrame({
    "img_id": test_data['img_id'].values, 
    "predicted_metastasis_ratio": y_pred.flatten()
})

submission.to_csv("submission.csv", index=False)

30/30 [==============================] - 4s 140ms/step
